# File Formats

There are many specialized file formats, used by many applications.  The Python standard library comes with support for a reasonable number of them, but for other purposes, various third-party libraries are needed.

Not all the formats that Python supports out of the box are discussed in this course, but many are in this series.  For example, internet-oriented formats like those used for email and HTML web pages are in other courses.  JSON and XML are discussed in the Data Serialization course.  A few others in the standard library are relatively uncommon or are old.

## Module: csv

The course on Data Serialization with Python deals with CSV in more detail.  But given the ubiquity of delimited files, we look at it very briefly here.  Let us look at a couple examples of reading from CSV, and one of writing to it.

In [1]:
import csv
from pprint import pprint

In [2]:
with open('Airline-Tweets.csv') as fh:
    airline = csv.reader(fh)
    headers = next(airline)
    for tweet in airline:
        for field, value, _ in zip(headers, tweet, range(6)):
            print(f"{field:>30}: {value}")
        break

                      tweet_id: 570306133677760513
             airline_sentiment: neutral
  airline_sentiment_confidence: 1.0
                negativereason: 
     negativereason_confidence: 
                       airline: Virgin America


In [3]:
with open('Airline-Tweets.csv') as fh:
    airline = csv.DictReader(fh, quoting=csv.QUOTE_MINIMAL)
    pprint(tweet := next(airline))

{'airline': 'Virgin America',
 'airline_sentiment': 'neutral',
 'airline_sentiment_confidence': '1.0',
 'airline_sentiment_gold': '',
 'name': 'cairdin',
 'negativereason': '',
 'negativereason_confidence': '',
 'negativereason_gold': '',
 'retweet_count': '0',
 'text': '@VirginAmerica What @dhepburn said.',
 'tweet_coord': '',
 'tweet_created': '2015-02-24 11:35:52 -0800',
 'tweet_id': '570306133677760513',
 'tweet_location': '',
 'user_timezone': 'Eastern Time (US & Canada)'}


Let's define some simple data that we would like to write out.

In [4]:
fields = ["Name", "Evaluation", "Rating", "Age"]
data = [
    ["Mia Johnson", "The movie was excellent", 9.5, 25],
    ["Liam Lopez", "Didn't really like it", 3.0, 35],
    ["Isabella Lee", "Wow! That was great", 8.0, 45]
]

We can write our in-memory data as a CSV file.

In [5]:
with open('tmp-movie.csv', 'w') as fh:
    movie = csv.writer(fh, quoting=csv.QUOTE_NONNUMERIC)
    for record in [fields]+data:
        movie.writerow(record)
        
!cat tmp-movie.csv

"Name","Evaluation","Rating","Age"
"Mia Johnson","The movie was excellent",9.5,25
"Liam Lopez","Didn't really like it",3.0,35
"Isabella Lee","Wow! That was great",8.0,45


## Module: pickle

The pickle format is a special, Python-specific, data format that serializes arbitrary Python objects.  It is discussed in much more detail in the Data Serialization course.  Almost any Python object can be written to disk and read back later (or shared, etc).

In [6]:
import pickle

Let us use the first tweet from the last section, which is represented as a dictionary.  The function `dump()` writes to a file while `dumps()` creates a bytestring.

In [7]:
with open('tmp-tweet.pkl', 'wb') as fh:
    pickle.dump(tweet, fh)

print(pickle.dumps(tweet))

b'\x80\x04\x95\xb6\x01\x00\x00\x00\x00\x00\x00}\x94(\x8c\x08tweet_id\x94\x8c\x12570306133677760513\x94\x8c\x11airline_sentiment\x94\x8c\x07neutral\x94\x8c\x1cairline_sentiment_confidence\x94\x8c\x031.0\x94\x8c\x0enegativereason\x94\x8c\x00\x94\x8c\x19negativereason_confidence\x94h\x08\x8c\x07airline\x94\x8c\x0eVirgin America\x94\x8c\x16airline_sentiment_gold\x94h\x08\x8c\x04name\x94\x8c\x07cairdin\x94\x8c\x13negativereason_gold\x94h\x08\x8c\rretweet_count\x94\x8c\x010\x94\x8c\x04text\x94\x8c#@VirginAmerica What @dhepburn said.\x94\x8c\x0btweet_coord\x94h\x08\x8c\rtweet_created\x94\x8c\x192015-02-24 11:35:52 -0800\x94\x8c\x0etweet_location\x94h\x08\x8c\ruser_timezone\x94\x8c\x1aEastern Time (US & Canada)\x94u.'


For reading we have `load()` versus `loads()` as well.

In [8]:
with open('tmp-tweet.pkl', 'rb') as fh:
    new = pickle.load(fh)
    pprint({k:v for (k,v) in new.items() if v})

{'airline': 'Virgin America',
 'airline_sentiment': 'neutral',
 'airline_sentiment_confidence': '1.0',
 'name': 'cairdin',
 'retweet_count': '0',
 'text': '@VirginAmerica What @dhepburn said.',
 'tweet_created': '2015-02-24 11:35:52 -0800',
 'tweet_id': '570306133677760513',
 'user_timezone': 'Eastern Time (US & Canada)'}


# Compression Formats

A number of standard library modules deal with the most popular compression techniques.

## Module: gzip

The `gzip` module is often used to handle files using the widespread `.gz` compression suffix.  The underlying file might be of any type; this format merely describes is manner of compression.

This module can actually compress within memory, and not only to and from files.  Sometimes this is useful for data being transmitted over the wire, for example.  Most of the time, data read from or written to files is what you care about.

In [9]:
import gzip
from pathlib import Path
tweets = Path('Airline-Tweets.csv')
gzfile = Path('tmp-tweets.csv.gz')

A quick look at the in-memory compression first.

In [10]:
data = b'A'*1_000_000
small = gzip.compress(data)
print("Bytes:", len(small))
print("Initial:", small[:20])

Bytes: 1004
Initial: b'\x1f\x8b\x08\x00/\xf0N_\x02\xff\xed\xc1\x81\x00\x00\x00\x00\xc3 \xb6'


Let us compress a file.  We use the large CSV of Tweets used above.

In [11]:
%%time
with gzip.open(gzfile, 'w') as gz:
    data = tweets.read_bytes()
    gz.write(data)

CPU times: user 220 ms, sys: 8.23 ms, total: 228 ms
Wall time: 228 ms


In [12]:
print(f"  Raw size: {tweets.stat().st_size:,}")
print(f"Compressed: {gzfile.stat().st_size:,}")

  Raw size: 3,421,431
Compressed: 1,118,042


## Module: bz2

The `bz2` module is very similar to the `gzip` module, just using a different underlying compression technique.  Most of the time, `.bz2` files are smaller than `.gz` equivalents, but take longer to compress and decompress.  Usually your concern will be dealing with the format you get, not weighing these trade-offs.

In [13]:
import bz2
bzfile = Path('tmp-tweets.csv.bz2')

A quick look at the in-memory compression first.

In [14]:
data = b'A'*1_000_000
small = bz2.compress(data)
print("Bytes:", len(small))
print("Initial:", small[:20])

Bytes: 48
Initial: b'BZh91AY&SY\x12\xa5~<\x00\x07\xa8\x84@\xa0'


Let us compress the large CSV of Tweets used above.

In [15]:
%%time
with bz2.open(bzfile, 'w') as bz:
    data = tweets.read_bytes()
    bz.write(data)

CPU times: user 503 ms, sys: 4.15 ms, total: 507 ms
Wall time: 506 ms


In [16]:
print(f"  Raw size: {tweets.stat().st_size:,}")
print(f"Compressed: {bzfile.stat().st_size:,}")

  Raw size: 3,421,431
Compressed: 796,184


## Module: lzma

The `lzma` module implements a still newer algorithm than `gzip` and `bz2`.  The tool `xz` and the older tool `lzma` use this format, as do a variety of other wrapping applications.  As a rule of thumb, `lzma` can often do slightly better than `bz2` in degree of compression; but is far slower than `gzip`, or even `bz2`.  Details vary with the kind of data you work with though.

In [17]:
import lzma
xzfile = Path('tmp-tweets.csv.xz')

A quick look at the in-memory compression first.

In [18]:
data = b'A'*1_000_000
small = lzma.compress(data)
print("Bytes:", len(small))
print("Initial:", small[:20])

Bytes: 276
Initial: b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x01\x16\x00\x00\x00'


Let us compress the large CSV of Tweets used above.

In [19]:
%%time
with lzma.open(xzfile, 'w', format=lzma.FORMAT_XZ) as xz:
    data = tweets.read_bytes()
    xz.write(data)

CPU times: user 2.12 s, sys: 56.3 ms, total: 2.18 s
Wall time: 2.18 s


In [20]:
print(f"  Raw size: {tweets.stat().st_size:,}")
print(f"Compressed: {xzfile.stat().st_size:,}")

  Raw size: 3,421,431
Compressed: 843,276


## Module: zipfile

The other compression modules we saw in the Python standard library are just about compression formats.  The module `zipfile` works with an actual archive format that might contain files compressed using any of the compression formats described above.

In [21]:
import zipfile
with zipfile.ZipFile('tmp-example.zip', 'w') as zf:
    zf.write('tmp-movie.csv')
    zf.write(tweets, compress_type=zipfile.ZIP_DEFLATED)
    zf.write('02-File-Formats.ipynb', 
                     compress_type=zipfile.ZIP_LZMA, 
                     compresslevel=9)

Usually you want to work with an existing zip archive.

In [22]:
with zipfile.ZipFile('tmp-example.zip') as zf:
    for info in zf.infolist():
        print(f"{info.filename:>22}: {info.file_size:,} ⟶ {info.compress_size:,}")

         tmp-movie.csv: 174 ⟶ 174
    Airline-Tweets.csv: 3,421,431 ⟶ 1,138,002
 02-File-Formats.ipynb: 24,841 ⟶ 4,927


For example, to work with one file inside the archive, just as if it were a plain file on disk:

In [23]:
with zipfile.ZipFile('tmp-example.zip') as zf:
    with zf.open('tmp-movie.csv') as movie:
        for line in movie:
            print(line.decode(), end='')

"Name","Evaluation","Rating","Age"
"Mia Johnson","The movie was excellent",9.5,25
"Liam Lopez","Didn't really like it",3.0,35
"Isabella Lee","Wow! That was great",8.0,45


## Module: sqlite3

SQLite is a lightweight disk-based database, where all tables, indices, etc. live inside a single file.  For the most part, the module `sqlite3` follows the same DB-API used to access multi-user RDBMSs within Python.  The sqlite3 data format is broadly used: for example, it is utilized on the largest supercompters for local storage and also on every iOS and Android device in the world as part of their operating system.

As an example, the same collection of tweets about airlines we have used earlier, was distributed by Kaggle in sqlite3 format along with CSV (with sqlite3 being the preferred choice).

In [24]:
import sqlite3
conn = sqlite3.connect('Airline-Tweets.sqlite')
cur = conn.cursor()

In [25]:
cur.execute('SELECT name FROM sqlite_master WHERE type="table"')
cur.fetchall()

[('Tweets',)]

In [26]:
cur.execute('SELECT * FROM Tweets LIMIT 2 OFFSET 1000')
for row in cur:
    print(row)

(567845681768968192, 'positive', 1, '', '', 'Southwest', '', 'beccalauren2011', '', 0, '@SouthwestAir I got it added thank you! :)', '', '2015-02-17 16:38:55 -0800', 'SMALL TOWN, USA', 'Central Time (US & Canada)')
(567845726220357632, 'neutral', 0.6601, '', '', 'Virgin America', '', 'josiebarosie', '', 0, '@VirginAmerica is that #thestarter??😁', '', '2015-02-17 16:39:05 -0800', 'Cork.Ireland', '')


It is sometimes friendlier to return dictionary-like rows as results to see what column names correspond to values.  This special `Row` object allows access by both column name and position.

In [27]:
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute('SELECT * FROM Tweets WHERE airline_sentiment="positive"')
row = cur.fetchone()
row.keys()

['tweet_id',
 'airline_sentiment',
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'airline_sentiment_gold',
 'name',
 'negativereason_gold',
 'retweet_count',
 'text',
 'tweet_coord',
 'tweet_created',
 'tweet_location',
 'user_timezone']

In [28]:
print(row['tweet_id'])
print(row[5:8])
dict(row)

567655489119326209
('Southwest', '', 'rjp1208')


{'tweet_id': 567655489119326209,
 'airline_sentiment': 'positive',
 'airline_sentiment_confidence': 1,
 'negativereason': '',
 'negativereason_confidence': '',
 'airline': 'Southwest',
 'airline_sentiment_gold': '',
 'name': 'rjp1208',
 'negativereason_gold': '',
 'retweet_count': 0,
 'text': '@SouthwestAir nice work on the update!',
 'tweet_coord': '',
 'tweet_created': '2015-02-17 04:03:09 -0800',
 'tweet_location': '',
 'user_timezone': 'Pacific Time (US & Canada)'}